# aiochris as a Shell

Using `aiochris` in either IPython or Jupyter can be an easy way to work with _ChRIS_ with code interactively.

## Getting Started

Install `aiochris` and `chrs` by running

```shell
pip install aiochris[chrs]
```

## Logging In

Run `chrs login` to log into _ChRIS_, or `chrs switch` to alternate between multiple _ChRIS_ user accounts and servers. Once you are logged in, you can
create a client using
[`ChrisClient.from_chrs`](https://fnndsc.github.io/aiochris/aiochris/client/authed.html#AuthenticatedClient.from_chrs).

In [1]:
from aiochris import ChrisClient
chris: ChrisClient = await ChrisClient.from_chrs()

What's great about `ChrisClient.from_chrs()` is that you will be able to share your notebook's code without sharing your _ChRIS_ login with them too.

## Searching for PACS Files

Let's try searching for all brain scans of female patients younger than 6 months old.
We are optionally using [`tqdm`](https://tqdm.github.io/) to show a progress bar for our search.

In [2]:
from tqdm.notebook import tqdm

search = chris.search_pacsfiles(
    PatientSex='F',          # female
    max_PatientAge=182,      # 6 months in days
    fname_icontains='BRAIN'  # file name includes "BRAIN"
)
series_dirs = set()
with tqdm(total=await search.count()) as pbar:
    async for pacs_file in search:
        series_dirs.add(pacs_file.parent)
        pbar.update()

  0%|          | 0/950 [00:00<?, ?it/s]

## Create a Feed

To analyze the data, we need to copy the files to a new feed.

In [3]:
from aiochris.models import Plugin, PluginInstance

pl_dircopy: Plugin = await chris.search_plugins(name='pl-dircopy').first()
copy_to_feed: PluginInstance = await pl_dircopy.create_instance(dir=','.join(series_dirs))
copy_to_feed.id

18

## Run a Plugin

Now that our data is in a feed, we can run plugins to process the data.
To help us, let's print out the parameters and help of our plugin first.

In [4]:
pl_dcm2niix: Plugin = await chris.search_plugins(name='pl-dcm2niix', version='0.1.0').get_only()
await pl_dcm2niix.print_help()

              b (-b): BIDS sidecar [choices: y, n, o] (default: y)
              d (-d): directory search depth. Convert DICOMs in sub-folders of in_folder? [choices: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9] (default: 5)
              f (-f): filename (%%a=antenna (coil) name, %%b=basename, %%c=comments, %%d=description, %%e=echo number, %%f=folder name, %%g=accession number, %%i=ID of patient, %%j=seriesInstanceUID, %%k=studyInstanceUID, %%m=manufacturer, %%n=name of patient, %%o=mediaObjectInstanceUID, %%p=protocol, %%r=instance number, %%s=series number, %%t=time, %%u=acquisition number, %%v=vendor, %%x=study ID; %%z=sequence name;) (default: %f_%p_%t_%s)
              m (-m): merge 2D slices from same series regardless of echo, exposure, etc. [no, yes, auto] [choices: n, y, 0, 1, 2] (default: 2)
              v (-v): verbose [no, yes, logorrheic] [choices: n, y, 0, 1, 2] (default: 0)
              x (-x): crop 3D acquisitions [choices: y, n, i] (default: n)
              z (-z): gz compress i

The parameters and previous plugin instance are passed to the
[`create_instance`](https://fnndsc.github.io/aiochris/aiochris/models/logged_in.html#Plugin.create_instance) method.

In [5]:
plinst: PluginInstance = await pl_dcm2niix.create_instance(previous=copy_to_feed, b='y', x='y')
plinst.id

19

You can wait for plugins to finish by calling [`PluginInstance.wait`](https://fnndsc.github.io/aiochris/models/logged_in.html#PluginInstance.wait)

In [6]:
time_elapsed, last_state = await plinst.wait()
print(f"Waited for {time_elapsed:.2f} seconds, status is {last_state.status}")

Waited for 110.67 seconds, status is Status.finishedSuccessfully
